<a href="https://colab.research.google.com/github/bclee232/vaccine-twitter-sentiment/blob/master/albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/lib/kaggle/gcp.py
/kaggle/input/bert-chinese/bert_model.ckpt.index
/kaggle/input/bert-chinese/bert_model.ckpt.data-00000-of-00001
/kaggle/input/bert-chinese/bert_model.ckpt.meta
/kaggle/input/snapshots/snapshot_model_1.h5
/kaggle/input/snapshots/snapshot_model_2.h5
/kaggle/input/snapshots/snapshot_model_3.h5
/kaggle/input/snapshots/snapshot_model_4.h5
/kaggle/input/snapshot/snapshot_model_1.h5
/kaggle/input/innovation/hydrated_data_training.csv
/kaggle/input/innovation/data_training.csv
/kaggle/input/innovation/hydrated_data_test.csv
/kaggle/working/__notebook_source__.ipynb


In [ ]:
data = pd.read_csv('/kaggle/input/innovation/data_training.csv')
hydrated_data = pd.read_csv('/kaggle/input/innovation/hydrated_data_training.csv')
test_data = pd.read_csv('/kaggle/input/innovation/hydrated_data_test.csv')

zero, one = data.tweet_label_int.value_counts()
print(zero, one, zero/(zero + one) * 100) # 88% 0 12% 1

tweet_label_int = data.tweet_label_int
tweet_label_list = tweet_label_int.tolist()
tweetid = data.tweetid
tweetid_list = tweetid.tolist()

hydrated_tweetid = hydrated_data.id
hydrated_tweetid_list = hydrated_tweetid.tolist()

labels = []

for i in range(len(hydrated_tweetid_list)):
    val = int(hydrated_tweetid_list[i])
    for j in range(len(tweetid_list)):
        if val == int(tweetid_list[j]):
            labels.append(float(tweet_label_list[j]))
            break

print(len(labels), labels[0:10])
hydrated_data['labels'] = labels
print(hydrated_data.shape)

7915 1085 87.94444444444444
8351 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(8351, 35)


In [ ]:
df = pd.DataFrame({'text': hydrated_data.text, 'tweet_label_int': labels, 'lang': hydrated_data.lang})

In [ ]:
data = df[df.lang == 'en'].reset_index()

In [ ]:
data

In [ ]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
# preprocessing
import re
import nltk
import spacy
import string

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_at(text):
    text = text.split()
    list = []
    for i in text:
        if "@" not in i:
            list.append(i)
    return ' '.join(list)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_hash(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', string.punctuation))

pre_data = data
pre_data["text"] = data["text"].apply(lambda text: expand_contractions(text))
pre_data["text"] = pre_data["text"].apply(lambda text: remove_at(text))
pre_data["text"] = pre_data["text"].apply(lambda text: remove_emoji(text))
pre_data["text"] = pre_data["text"].apply(lambda text: remove_urls(text))
pre_data["text"] = pre_data["text"].apply(lambda text: remove_hash(text))

In [ ]:
pre_data.head()

In [ ]:
train_labels = np.array(hydrated_data['labels']).astype('int')
print(train_labels.shape)

In [ ]:
pre_test = pd.DataFrame({'text': test_data.text, 'lang': test_data.lang})
pre_test

In [ ]:
pre_test["text"] = pre_test["text"].apply(lambda text: expand_contractions(text))
pre_test["text"] = pre_test["text"].apply(lambda text: remove_at(text))
pre_test["text"] = pre_test["text"].apply(lambda text: remove_emoji(text))
pre_test["text"] = pre_test["text"].apply(lambda text: remove_urls(text))
pre_test["text"] = pre_test["text"].apply(lambda text: remove_hash(text))

In [ ]:
pre_test

In [ ]:
train_data_text = hydrated_data['text'].astype(str)
test_data_text = test_data['text'].astype(str)
print(train_data_text, test_data_text)

In [ ]:
def meta_text(text):
    data_text_len = [len(x) for x in text]
    return (data_text_len[np.argmax(data_text_len)], data_text_len[np.argmin(data_text_len)], np.mean(data_text_len))
print("train", meta_text(train_data_text), "test", meta_text(test_data_text)) # max, min, mean

In [ ]:
zero, one = np.bincount(train_labels)
total_count = len(train_labels)
weight_zero = (1 / zero) * (total_count) / 2.0
weight_one = (1 / one) * (total_count) / 2.0
class_weights = {0: weight_zero, 1: weight_one}

In [ ]:
print(total_count, zero, one)
print(class_weights)

In [ ]:
import transformers
t_albert = transformers.AlbertTokenizer.from_pretrained('albert-large-v2', do_lower_case=True) # resource exhausted using xxlarge
albert_model = transformers.TFAlbertModel.from_pretrained('albert-large-v2')

In [ ]:
def albert_encode(data, max_len, t):
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = t.encode_plus(data[i],
                               add_special_tokens=True,
                               max_length=max_len,
                               pad_to_max_length=True,
                               return_attention_mask=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = albert_encode(train_data_text, 100, t_albert) # 100
x_train = [train_input_ids[1600:], train_attention_masks[1600:]]
y_train = train_labels[1600:]
x_val = [train_input_ids[:1600], train_attention_masks[:1600]]
y_val = train_labels[:1600]
test_input_ids, test_attention_masks = albert_encode(test_data_text, 100, t_albert)

In [ ]:
import keras.backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
from keras import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam

from keras.layers import LSTM,  Embedding, BatchNormalization, Dense, TimeDistributed, Dropout, Bidirectional, Flatten, GlobalMaxPool1D

def create_biLSTM(albert_model):
    
    input_ids = Input(shape=(100,), dtype='int32')
    attention_masks = Input(shape=(100,), dtype='int32')
    
    output = albert_model([input_ids, attention_masks])
    output = output[0] # get embeddings - not [1]
    output = Bidirectional(LSTM(32, dropout=0.1, recurrent_dropout=0.5))(output) # changed to 16
    output = Dense(1, activation='sigmoid')(output)

    model = Model(inputs = [input_ids, attention_masks], outputs=output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=[f1_score])
    
    return model

def create_model(albert_model):
    input_ids = Input(shape=(100,),dtype='int32')
    attention_masks = Input(shape=(100,), dtype='int32')
    
    output = albert_model([input_ids, attention_masks])
    output = output[1]
    output = Dense(1, activation='sigmoid')(output)
    model = Model(inputs = [input_ids, attention_masks], outputs=output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=[f1_score])
    
    return model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
import h5py

es = EarlyStopping(monitor='val_f1_score',mode='max',verbose=1, patience=5)
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_f1_score', verbose=1, save_best_only=True, mode='max', period=1)
callbacks = [es, checkpoint]

In [ ]:
model = create_biLSTM(albert_model)
model.summary()

In [ ]:
from keras.callbacks import Callback
from math import pi
from math import cos
from math import floor
from keras import backend

# snapshot ensemble with custom learning rate schedule
class SnapshotEnsemble(Callback):
    # constructor
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()
 
    # calculate learning rate for epoch
    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = floor(n_epochs/n_cycles)
        cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (cos(cos_inner) + 1)
 
    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs={}):
        # calculate learning rate
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        # set learning rate
        backend.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrates.append(lr)
 
    # save models at the end of each cycle
    def on_epoch_end(self, epoch, logs={}):
        # check if we can save model
        epochs_per_cycle = floor(self.epochs / self.cycles)
        if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
            # save model to file
            filename = "/kaggle/working/snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
            self.model.save_weights(filename)
            print('>saved snapshot %s, epoch %d' % (filename, epoch))

In [ ]:
# create snapshot ensemble callback
n_epochs = 20
n_cycles = n_epochs / 5
ca = SnapshotEnsemble(n_epochs, n_cycles, 0.01)

In [ ]:
x_train = [train_input_ids[1600:], train_attention_masks[1600:]]
y_train = train_labels[1600:]
x_val = [train_input_ids[:1600], train_attention_masks[:1600]]
y_val = train_labels[:1600]

In [ ]:
hist = model.fit(x_train, 
                 y_train,
                 epochs=n_epochs,
                 batch_size=32,
                 class_weight=class_weights,
                validation_data=(x_val, y_val),
                callbacks=[ca])

In [ ]:
"""hist = model.fit([train_input_ids, train_attention_masks], 
                 train_labels,
                 epochs=1,
                 batch_size=32,
                 validation_split=0.2,
                 class_weight=class_weights,
                callbacks=callbacks)""" # 14

In [ ]:
# load models and make a snapshot ensemble prediction
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
import numpy
from numpy import array
from numpy import argmax

# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'snapshot_model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = array(yhats)
    # sum across ensemble members
    summed = numpy.sum(yhats, axis=0)
    # argmax across classes
    result = argmax(summed, axis=1)
    return result

# evaluate a specific number of members in an ensemble
def evaluate_n_members(members, n_members, testX, testy):
    # select a subset of members
    subset = members[:n_members]
    # make prediction
    yhat = ensemble_predictions(subset, testX)
    # calculate accuracy
    return accuracy_score(testy, yhat)

In [ ]:
result = model.predict([test_input_ids, test_attention_masks])
result = np.round(result).astype(int)
res_list = result.flatten().tolist()
output = pd.DataFrame({'tweetid': test_data.id, 'tweet_label_int': res_list})
output.to_csv('submission.csv', index=False)